# **Phase 4**

In [ ]:
!pip install keras-tuner seaborn xgboost

In [ ]:
#some of the imports may be repeated
import sklearn
print(sklearn.__version__)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix, f1_score
from sklearn.impute import SimpleImputer
import seaborn as sb
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
import xgboost as xgb
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import keras_tuner as kt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.svm import SVC
from sklearn.cluster import DBSCAN
from sklearn.metrics import adjusted_rand_score
from google.colab import drive

In [ ]:
#some of the code blocks may be repeated

drive.mount('/content/drive')
df = pd.read_csv('/path/to/csv/cleaned_dataset.csv')

df = df.sample(frac=0.25, random_state=42)
df = df.sample(frac=0.25, random_state=42)

# If you need to sample the data
# df = df.sample(frac=0.25, random_state=42)
# df = df.sample(frac=0.25, random_state=42)

#Feature Encoding
df['CRASH DATE'] = pd.to_datetime(df['CRASH DATE'])
df['DayOfWeek'] = df['CRASH DATE'].dt.dayofweek  # Monday=0, Sunday=6
df['CRASH TIME'] = pd.to_datetime(df['CRASH TIME'], format='%H:%M')
df['HourOfDay'] = df['CRASH TIME'].dt.hour  # Extract hour (0-23)

df = pd.get_dummies(df, columns=['BOROUGH'], drop_first=True)  # One-hot encoding (give values 0 or 1 to the categorical values) for boroughs
df['ZIP CODE'] = pd.to_numeric(df['ZIP CODE'], errors='coerce')  # zip to numeric



for col in ['CONTRIBUTING FACTOR VEHICLE 1', 'CONTRIBUTING FACTOR VEHICLE 2', 'CONTRIBUTING FACTOR VEHICLE 3']:
    df[col].fillna("Unknown", inplace=True)  # missing values marked as "Unknown"
    df[col] = LabelEncoder().fit_transform(df[col])



df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
df.replace('', float('nan'), inplace=True)

df['ZIP CODE'] = pd.to_numeric(df['ZIP CODE'], errors='coerce')

features_clustering = [
    'NUMBER OF PERSONS INJURED', 'NUMBER OF PERSONS KILLED',
    'NUMBER OF PEDESTRIANS INJURED', 'NUMBER OF PEDESTRIANS KILLED',
    'NUMBER OF CYCLIST INJURED', 'NUMBER OF CYCLIST KILLED',
    'NUMBER OF MOTORIST INJURED', 'NUMBER OF MOTORIST KILLED',
    'DayOfWeek', 'HourOfDay',
    'CONTRIBUTING FACTOR VEHICLE 1', 'CONTRIBUTING FACTOR VEHICLE 2',
    'CONTRIBUTING FACTOR VEHICLE 3'
]

#ensure df_filtered has only these 13 columns
df_filtered = df[features_clustering].dropna()


scaler = StandardScaler()
df_scaled = scaler.fit_transform(df_filtered)

In [ ]:
df['SEVERE_ACCIDENT'] = (df['NUMBER OF PERSONS INJURED'] > 0) | (df['NUMBER OF PERSONS KILLED'] > 0)
df['SEVERE_ACCIDENT'] = df['SEVERE_ACCIDENT'].astype(int)

df['BOROUGH'] = df['BOROUGH_BROOKLYN'] | df['BOROUGH_MANHATTAN'] | df['BOROUGH_QUEENS'] | df['BOROUGH_STATEN ISLAND']

df['CRASH TIME'] = pd.to_datetime(df['CRASH TIME'])
df['CRASH HOUR'] = df['CRASH TIME'].dt.hour

features = [
    'CONTRIBUTING FACTOR VEHICLE 1', 'BOROUGH', 'CRASH HOUR',
    'VEHICLE TYPE CODE 1', 'LATITUDE', 'LONGITUDE'
]
X = df[features]
y = df['SEVERE_ACCIDENT']

# Preprocessing pipeline
numeric_features = ['CRASH HOUR', 'LATITUDE', 'LONGITUDE']
categorical_features = ['CONTRIBUTING FACTOR VEHICLE 1', 'BOROUGH', 'VEHICLE TYPE CODE 1']

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ])

X_processed = preprocessor.fit_transform(X)

# Split data
X_train, X_test, y_train, y_test = train_test_split(X_processed, y, test_size=0.2, random_state=42)


In [ ]:
# ────────────────────────────────────────────────────────────────
# 1.  Load data & inspect columns #this would help KMeans
# ────────────────────────────────────────────────────────────────
import pandas as pd
from google.colab import drive

drive.mount('/content/drive')
RAW_PATH = "/path/to/csv/cleaned_dataset.csv"
df = pd.read_csv(RAW_PATH)

df = df.sample(frac=0.25, random_state=42)
df = df.sample(frac=0.25, random_state=42)

print("🗒️ Columns in df:")
for col in df.columns:
    print("  •", col)

# If you have 'HourOfDay' instead of 'CRASH HOUR', rename it
if 'HourOfDay' in df.columns and 'CRASH HOUR' not in df.columns:
    df['CRASH HOUR'] = df['HourOfDay']
    print("✅ Renamed 'HourOfDay' → 'CRASH HOUR'")

# ────────────────────────────────────────────────────────────────
# 2.  Feature engineering & target
# ────────────────────────────────────────────────────────────────
df['CRASH DATE'] = pd.to_datetime(df['CRASH DATE'])
df['DayOfWeek'] = df['CRASH DATE'].dt.dayofweek

df['CRASH TIME'] = pd.to_datetime(df['CRASH TIME'], format='%H:%M', errors='coerce')
df['HourOfDay'] = df['CRASH TIME'].dt.hour  # backup
df['CRASH HOUR'] = df['CRASH TIME'].dt.hour  # ensure this exists

df['ZIP CODE'] = pd.to_numeric(df['ZIP CODE'], errors='coerce')

# Fill & label‐encode contributing factors
from sklearn.preprocessing import LabelEncoder
for col in ['CONTRIBUTING FACTOR VEHICLE 1',
            'CONTRIBUTING FACTOR VEHICLE 2',
            'CONTRIBUTING FACTOR VEHICLE 3']:
    df[col].fillna("Unknown", inplace=True)
    df[col] = LabelEncoder().fit_transform(df[col])

# Create binary target
df['SEVERE_ACCIDENT'] = (
    (df['NUMBER OF PERSONS INJURED'] > 0) |
    (df['NUMBER OF PERSONS KILLED'] > 0)
).astype(int)

# ────────────────────────────────────────────────────────────────
# 3.  Persist standalone borough encoder #one hot encoding for standardization
# ────────────────────────────────────────────────────────────────
from sklearn.preprocessing import OneHotEncoder
import joblib

borough_ohe = OneHotEncoder(
    sparse_output=False,
    handle_unknown="ignore"
)
borough_ohe.fit(df[['BOROUGH']])
joblib.dump(borough_ohe, 'borough_encoder.pkl')
print("✅ Saved borough_encoder.pkl with:", borough_ohe.categories_[0])

# ────────────────────────────────────────────────────────────────
# 4.  Build & save full preprocessor
# ────────────────────────────────────────────────────────────────
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler

numeric_feats     = ['CRASH HOUR', 'LATITUDE', 'LONGITUDE']
categorical_feats = ['CONTRIBUTING FACTOR VEHICLE 1', 'BOROUGH', 'VEHICLE TYPE CODE 1']

preprocessor = ColumnTransformer(transformers=[
    ("num", StandardScaler(), numeric_feats),
    ("cat", OneHotEncoder(sparse_output=False, handle_unknown="ignore"),
     categorical_feats),
])

# Fit on raw X (with BOROUGH as string)
preprocessor.fit(df[numeric_feats + categorical_feats])
joblib.dump(preprocessor, 'preprocessor.pkl')
print("✅ Saved preprocessor.pkl (BOROUGH categories: ",
      preprocessor.named_transformers_['cat'].categories_[1], ")")

# ────────────────────────────────────────────────────────────────
# 5.  Transform features and split for modeling #this is how we prevent overfitting
# ────────────────────────────────────────────────────────────────
from sklearn.model_selection import train_test_split

X = df[numeric_feats + categorical_feats]
y = df['SEVERE_ACCIDENT']

X_proc = preprocessor.transform(X)
X_train, X_test, y_train, y_test = train_test_split(
    X_proc, y, test_size=0.2, random_state=42
)

print("✅ Data ready: X_train.shape =", X_train.shape, ", y_train.mean() =", y_train.mean())


In [ ]:
# ────────────────────────────────────────────────────────────────
# XGBoost
# ────────────────────────────────────────────────────────────────
from sklearn.impute import SimpleImputer
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score
import xgboost as xgb
import matplotlib.pyplot as plt
import seaborn as sns

#Impute any missing values
imputer = SimpleImputer(strategy='mean')
X_train_imputed = imputer.fit_transform(X_train)
X_test_imputed  = imputer.transform(X_test)

#Set up grid search
param_grid_xgb = {
    'n_estimators':    [50, 100, 200],
    'learning_rate':   [0.01, 0.1, 0.2],
    'max_depth':       [3, 5, 7],
}

xgb_model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss')
grid_xgb  = GridSearchCV(
    xgb_model,
    param_grid_xgb,
    cv=5,
    scoring='accuracy',
    n_jobs=-1
)

#Fit & select best
grid_xgb.fit(X_train_imputed, y_train)
best_xgb = grid_xgb.best_estimator_
xgb_model = best_xgb  # ensure we save the fitted model

#Predict & evaluate
y_pred_xgb    = best_xgb.predict(X_test_imputed)
confusion_xgb = confusion_matrix(y_test, y_pred_xgb)

print("XGBoost Best Parameters:", grid_xgb.best_params_)
print("XGBoost Accuracy:", accuracy_score(y_test, y_pred_xgb))
print("XGBoost F1 Score:", f1_score(y_test, y_pred_xgb))

#Plotting confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(confusion_xgb, annot=True, fmt='d', cmap='Purples')
plt.title('XGBoost Confusion Matrix')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()


In [ ]:
#KMEANS
inertia = []
K_range = range(2, 10)

for k in K_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(df_scaled)
    inertia.append(kmeans.inertia_)

# Plot the elbow graph
plt.plot(K_range, inertia, marker='o')
plt.xlabel('Number of Clusters')
plt.ylabel('Inertia')
plt.title('Elbow Method for Optimal K')
plt.show()

# Reduce dimensions to 2D for visualization
#pca = PCA(n_components=2)
#df_pca = pca.fit_transform(df_scaled)

k_range = range(3, 10)
scores = []

for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    labels = kmeans.fit_predict(df_scaled)
    score = silhouette_score(df_scaled, labels)
    scores.append({k: score})
    print(f"Silhouette score for k={k}: {score:.3f}")

optimal_k = max(scores, key=lambda x: list(x.values())[0])
optimal_k = list(optimal_k.keys())[0]
print(f"Optimal k: {optimal_k}")

kmeans = KMeans(n_clusters=optimal_k, random_state=42, n_init=10)
kmeans.fit(df_scaled)

# Reduce dimensions to 2D for visualization
pca = PCA(n_components=2)
df_pca = pca.fit_transform(df_scaled)

# Add cluster labels to PCA dataframe
df_plot = pd.DataFrame(df_pca, columns=['PC1', 'PC2'])
df_plot['Cluster'] = kmeans.labels_

# Plot the clusters
plt.figure(figsize=(10, 8))
for cluster in range(optimal_k):
    cluster_data = df_plot[df_plot['Cluster'] == cluster]
    plt.scatter(cluster_data['PC1'], cluster_data['PC2'], label=f'Cluster {cluster}')

plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.title('Clusters Visualization')
plt.legend()
plt.show()


In [ ]:
!pip install --quiet streamlit pyngrok joblib

In [ ]:
import joblib
from google.colab import files
# after training...
# joblib.dump(preprocessor,   "preprocessor.pkl")
joblib.dump(xgb_model,      "xgb_model.pkl")
# joblib.dump(kmeans,      "kmeans.pkl")

In [ ]:
# ────────────────────────────────────────────────────────────────
# (Re)building and save a standalone borough encoder #this would be effective for a single model
# ────────────────────────────────────────────────────────────────
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
import joblib
import streamlit as st

# path to your raw DataFrame CSV
DATA_PATH = '/path/to/csv/cleaned_dataset.csv'

# Load raw data and fit an OHE on the BOROUGH column
df = pd.read_csv(DATA_PATH)
borough_ohe = OneHotEncoder(sparse_output=False, handle_unknown="ignore")
borough_ohe.fit(df[["BOROUGH"]])
# Persist it
joblib.dump(borough_ohe, "borough_encoder.pkl")
print("Created borough_encoder.pkl with:", borough_ohe.categories_[0])

# ────────────────────────────────────────────────────────────────
# 1.  Debug: inspect your main preprocessor + new borough encoder
# ────────────────────────────────────────────────────────────────
# Force Streamlit to clear any cached models
st.cache_data.clear()

# Reload artifacts
preprocessor = joblib.load("preprocessor.pkl")
xgb_model    = joblib.load("xgb_model.pkl")

# Reconstruct feature lists
feature_names = list(preprocessor.feature_names_in_)
def flatten(keys):
    if isinstance(keys, (list, tuple)):
        out = []
        for k in keys:
            out.extend(flatten(k))
        return out
    return [keys]

num_keys = flatten(preprocessor.transformers_[0][2])
cat_keys = flatten(preprocessor.transformers_[1][2])
numeric_names     = [feature_names[i] if isinstance(i,int) else i for i in num_keys]
categorical_names = [feature_names[i] if isinstance(i,int) else i for i in cat_keys]

# Your pipeline’s OHE
ohe = preprocessor.named_transformers_['cat']

# Standalone borough encoder you just built
borough_enc = joblib.load("borough_encoder.pkl")

# Print to console
print("🔹 Numeric features:", numeric_names)
print("🔹 Categorical features:", categorical_names)
print("🔹 Pipeline OHE categories per feature:")
for name, cats in zip(categorical_names, ohe.categories_):
    print(f"   • {name}: {cats}")
print("🔹 Standalone borough_encoder categories:", borough_enc.categories_[0])

# Minimal Streamlit output to trigger a run
st.write("✅ Debug info printed to your console. Check your terminal for the full list.")


In [ ]:
# ────────────────────────────────────────────────────────────────
# PREPARING CLUSTERING DATA IN‐MEMORY (no extra files)
# ────────────────────────────────────────────────────────────────
import pandas as pd
import numpy as np
import joblib
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA

# 1) Load & downsample exactly as before
df = pd.read_csv("/path/to/csv/cleaned_dataset.csv")
df = df.sample(frac=0.25, random_state=42).sample(frac=0.25, random_state=42)

# 2) Feature engineering
df['CRASH TIME'] = pd.to_datetime(df['CRASH TIME'], format='%H:%M', errors='coerce')
df['CRASH HOUR'] = df['CRASH TIME'].dt.hour

df['BOROUGH'].fillna("Unknown", inplace=True)
df['LATITUDE'].fillna(df['LATITUDE'].mean(), inplace=True)
df['LONGITUDE'].fillna(df['LONGITUDE'].mean(), inplace=True)

for col in [
    'CONTRIBUTING FACTOR VEHICLE 1',
    'CONTRIBUTING FACTOR VEHICLE 2',
    'CONTRIBUTING FACTOR VEHICLE 3'
]:
    df[col].fillna("Unknown", inplace=True)
    df[col] = LabelEncoder().fit_transform(df[col])

# 3) Build raw feature matrix
features = [
    'CONTRIBUTING FACTOR VEHICLE 1',
    'BOROUGH',
    'CRASH HOUR',
    'VEHICLE TYPE CODE 1',
    'LATITUDE',
    'LONGITUDE'
]
X_raw = df[features]

# 4) Transform via your saved preprocessor
preprocessor = joblib.load("preprocessor.pkl")
X_proc       = preprocessor.transform(X_raw)

# 5) Drop any residual NaNs
mask    = ~np.isnan(X_proc).any(axis=1)
X_clean = X_proc[mask]

# 6) Run PCA → 2 components
pca   = PCA(n_components=2)
X_pca = pca.fit_transform(X_clean)

# 7) Keep X_pca in the notebook’s namespace for Streamlit to pick up
print("✅ Prepared X_pca in‐memory (shape", X_pca.shape, ")")

In [ ]:
app_code = """\
import streamlit as st
import pandas as pd
import numpy as np
import joblib
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.decomposition import PCA

# ────────────────────────────────────────────────────────────────
# 1) Load prediction artifacts & KMeans
# ────────────────────────────────────────────────────────────────
@st.cache_data
def load_models():
    preprocessor    = joblib.load("preprocessor.pkl")     # for severity
    xgb_model       = joblib.load("xgb_model.pkl")
    kmeans          = joblib.load("kmeans.pkl")      # trained on df_scaled
    borough_encoder = joblib.load("borough_encoder.pkl")
    return preprocessor, xgb_model, kmeans, borough_encoder

preprocessor, xgb_model, kmeans, borough_encoder = load_models()

# ────────────────────────────────────────────────────────────────
# 2) Re-create your original df_scaled & PCA coordinates
# ────────────────────────────────────────────────────────────────
@st.cache_data
def build_clustering_data():
    df = pd.read_csv("/path/to/csv/cleaned_dataset.csv")

    # Downsample exactly as before
    df = df.sample(frac=0.25, random_state=42).sample(frac=0.25, random_state=42)

    # Create target and drop unused columns
    df['CRASH TIME'] = pd.to_datetime(df['CRASH TIME'], format='%H:%M', errors='coerce')
    df['CRASH HOUR'] = df['CRASH TIME'].dt.hour

    # Fill and encode factors
    for col in ['CONTRIBUTING FACTOR VEHICLE 1',
                'CONTRIBUTING FACTOR VEHICLE 2',
                'CONTRIBUTING FACTOR VEHICLE 3']:
        df[col] = df[col].fillna("Unknown")
        df[col] = LabelEncoder().fit_transform(df[col])

    # Define exactly the clustering features
    features_clustering = [
        'NUMBER OF PERSONS INJURED', 'NUMBER OF PERSONS KILLED',
        'NUMBER OF PEDESTRIANS INJURED', 'NUMBER OF PEDESTRIANS KILLED',
        'NUMBER OF CYCLIST INJURED', 'NUMBER OF CYCLIST KILLED',
        'NUMBER OF MOTORIST INJURED', 'NUMBER OF MOTORIST KILLED',
        'DayOfWeek', 'HourOfDay',
        'CONTRIBUTING FACTOR VEHICLE 1', 'CONTRIBUTING FACTOR VEHICLE 2',
        'CONTRIBUTING FACTOR VEHICLE 3'
    ]

    # Make sure these exist
    df['DayOfWeek'] = pd.to_datetime(df['CRASH DATE']).dt.dayofweek
    df['HourOfDay'] = df['CRASH TIME'].dt.hour

    df_filtered = df[features_clustering].dropna()

    # Scale just like you did in training
    scaler = StandardScaler()
    df_scaled = scaler.fit_transform(df_filtered)

    # PCA down to 2D
    pca = PCA(n_components=2)
    df_pca = pca.fit_transform(df_scaled)

    return df_scaled, df_pca

X_scaled, X_pca = build_clustering_data()

# ────────────────────────────────────────────────────────────────
# 3) Build multi‐view UI
# ────────────────────────────────────────────────────────────────
st.title("🚦 NYC Crash Analyzer")
tab1, tab2 = st.tabs(["Severity Prediction", "Cluster Explorer"])

# Metadata for severity prediction
feature_names     = list(preprocessor.feature_names_in_)
cat_transformer   = preprocessor.named_transformers_['cat']
categorical_names = list(cat_transformer.feature_names_in_)
borough_list      = list(borough_encoder.categories_[0])

# ────────────────────────────────────────────────────────────────
with tab1:
    st.header("🔍 Accident Severity Predictor")
    st.sidebar.header("Input Features")

    inputs = {}
    for feat in feature_names:
        if feat == "BOROUGH":
            inputs[feat] = st.sidebar.selectbox("Borough", borough_list, key=f"cat_{feat}")
        elif feat in categorical_names:
            idx  = categorical_names.index(feat)
            opts = cat_transformer.categories_[idx]
            inputs[feat] = st.sidebar.selectbox(feat, opts, key=f"cat_{feat}")
        else:
            if feat == "CRASH HOUR":
                inputs[feat] = st.sidebar.slider(feat, 0, 23, 12, key=f"num_{feat}")
            else:
                inputs[feat] = st.sidebar.number_input(feat, value=0.0, key=f"num_{feat}")

    X_df   = pd.DataFrame([inputs], columns=feature_names)
    X_test = preprocessor.transform(X_df)
    pred   = xgb_model.predict(X_test)[0]
    proba  = xgb_model.predict_proba(X_test)[0]

    st.markdown(f"**Predicted severity:** `{pred}`")
    st.dataframe(
        pd.DataFrame([proba], columns=[f"Class {c}" for c in xgb_model.classes_]),
        use_container_width=True
    )

# ────────────────────────────────────────────────────────────────
with tab2:
    st.header("📊 Cluster Explorer")

    x_axis = st.selectbox("X-axis", ["PC1", "PC2"], index=0)
    y_axis = st.selectbox("Y-axis", ["PC1", "PC2"], index=1)
    axis_idx = {"PC1": 0, "PC2": 1}
    xi, yi   = axis_idx[x_axis], axis_idx[y_axis]

    # Correctly predict on the 13-d scaled data
    labels = kmeans.predict(X_scaled)

    plot_df = pd.DataFrame({
        x_axis: X_pca[:, xi],
        y_axis: X_pca[:, yi],
        "Cluster": labels.astype(str)
    })

    fig, ax = plt.subplots()
    for cl in sorted(plot_df["Cluster"].unique()):
        subset = plot_df[plot_df["Cluster"] == cl]
        ax.scatter(subset[x_axis], subset[y_axis], label=f"Cluster {cl}", alpha=0.6)
    ax.set_xlabel(x_axis)
    ax.set_ylabel(y_axis)
    ax.legend()
    st.pyplot(fig)
"""

# write out the file
with open("app.py", "w") as f:
    f.write(app_code)

print("app.py has been updated with borough dropdowns and improved UI")


In [ ]:
!ngrok authtoken 2veWgSMKSZUXoq3cDHk6gj7KPy2_2Soz335NRQEtrGw9tLcwC

In [ ]:
#Exposingg Streamlit’s port (8501) via ngrok
from pyngrok import ngrok

# Terminate any previous tunnels
ngrok.kill()

# Open a new tunnel
public_url = ngrok.connect(8501, "http")
print("→ Streamlit will be available at:", public_url)


In [ ]:
!nohup streamlit run app.py --server.port 8501 > streamlit.log 2>&1 &

In [ ]:
!pkill ngrok

In [ ]:
!streamlit cache clear

In [ ]:
import joblib, inspect, xgboost as xgb
obj = joblib.load("xgb_model.pkl")
print(type(obj))
if isinstance(obj, xgb.XGBClassifier):
    print("Booster attached:", hasattr(obj, "_Booster"))
    print("n_features_in_  :", getattr(obj, "n_features_in_", None))
